
# Setup Argilla with Hugging Face



In [ ]:
!pip install argilla huggingface_hub datasets -qqq

In [ ]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|
    
    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To login, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Token: 
Add token as git credential? (Y/n) n
Token is valid (permission: write).
Your token has been saved to /roo

In [ ]:
import argilla as rg
import pandas as pd

running Argilla using [Hugging Face Spaces](https://docs.argilla.io/en/latest/getting_started/installation/deployments/huggingface-spaces.html), init the Argilla client with the `ARGILLA_API_URL` and`ARGILLA_API_KEY`:

In [ ]:
# Replace api_url with the url to your HF Spaces URL if using Spaces
# Replace api_key if you configured a custom API key
rg.init(
    api_url=r"https://mansiuniyal-indic-translation.hf.space",
    api_key="admin.apikey"
)

/usr/local/lib/python3.10/dist-packages/argilla/client/client.py:165: UserWarning: No workspace configuration was detected. To work with Argilla datasets, specify a valid workspace name on `rg.init` or set it up through the `rg.set_workspace` function.
  warnings.warn(


# Create Dataset

[create a FeedbackDataset](https://docs.argilla.io/en/latest/guides/llms/practical_guides/create_dataset.html) with custom configuration for collecting feedback.

In [ ]:
# can be edited
lang = ["hindi","punjabi","gujrati","bengali","marwadi","tamil","kannad","assami"]
ds_emotion = rg.FeedbackDataset(
    fields=[
        rg.TextField(name="text")
    ],
    questions=[
        rg.LabelQuestion(name="translated-lang", labels=lang),
        rg.TextQuestion(name="translated-text")
    ],
    guidelines="English gender neutral dataset: translation into various Indic langugaes."
)


In [ ]:
file_path = "/content/3k_gender_neutral_data.csv"

dataset = pd.read_csv(file_path)
dataset.loc[0,'text']

'wow ! i do love a good horror movie . loving this cooler weather'

In [ ]:
records = []
for row in range(len(dataset)):
  record = rg.FeedbackRecord(
      fields={"text": dataset.loc[row,'text']}
  )
  records.append(record)

ds_emotion.add_records(records)
ds_emotion.push_to_argilla(name="translation", workspace="admin")

Pushing records to Argilla...: 100%|██████████| 102/102 [00:18<00:00,  5.40it/s]


<FeedbackDataset id=ec9ff51c-e95b-4b61-8e69-883cc2ffc33c name=translation workspace=Workspace(id=8c2e2958-9953-4aed-a9a3-399538ae5044, name=admin, inserted_at=2023-10-08 05:28:07, updated_at=2023-10-08 05:28:07) url=https://mansiuniyal-indic-translation.hf.space/dataset/ec9ff51c-e95b-4b61-8e69-883cc2ffc33c/annotation-mode fields=[TextField(id=UUID('6e2b48c5-2861-4046-8196-25e36c13830f'), name='text', title='Text', required=True, type='text', settings={'type': 'text', 'use_markdown': False}, use_markdown=False)] questions=[LabelQuestion(id=UUID('e8c62cdc-b793-4af7-a137-ea241b1738dc'), name='translated-lang', title='Translated-lang', description=None, required=True, type='label_selection', settings={'type': 'label_selection', 'options': [{'value': 'hindi', 'text': 'hindi'}, {'value': 'punjabi', 'text': 'punjabi'}, {'value': 'gujrati', 'text': 'gujrati'}, {'value': 'bengali', 'text': 'bengali'}, {'value': 'marwadi', 'text': 'marwadi'}, {'value': 'tamil', 'text': 'tamil'}, {'value': 'kanna

Load your annotations and safely store your annotations to the hub only after annotation is completed!

# Load dataset

save to hugging face spaces and then reloading from the Hugging Face hub

In [ ]:
ds_emotion_with_annotations = rg.FeedbackDataset.from_argilla(name="translation", workspace="admin")
ds_emotion_with_annotations.push_to_huggingface(repo_id="mansiuniyal/translation")

In [ ]:
ds_emotion = rg.FeedbackDataset.from_huggingface("mansiuniyal/translation", split="train[:100]")
ds_emotion.push_to_argilla(name="translate", workspace="admin")